In [2]:
import requests
from lxml import html

In [3]:
import datetime

# Menetapkan tanggal awal dan tanggal akhir
start_date = datetime.date(2023, 12, 30)
end_date = datetime.date(2023, 12, 31)

# Base URL tanpa parameter tanggal
base_url = 'https://www.jpnn.com/indeks?id=&d={}&m={}&y={}'

# Iterasi mundur dari tanggal awal ke tanggal akhir
current_date = end_date

formatted_url = []

while current_date >= start_date:
    # Membuat URL dengan menambahkan tanggal yang diformat
    formatted_url.append(base_url.format(current_date.day, current_date.month, current_date.year))
    
    # Mengurangi satu hari
    current_date -= datetime.timedelta(days=1)

In [4]:
formatted_url[0:10]

['https://www.jpnn.com/indeks?id=&d=31&m=12&y=2023',
 'https://www.jpnn.com/indeks?id=&d=30&m=12&y=2023']

In [5]:
import sqlite3

# Membuat koneksi ke database SQLite
conn = sqlite3.connect('./news_data.db')
c = conn.cursor()

# Membuat tabel baru jika belum ada
c.execute('''
CREATE TABLE IF NOT EXISTS news (
    judul TEXT,
    url TEXT,
    rubrik TEXT,
    tanggal TEXT
)
''')
conn.commit()

In [8]:
from tqdm import tqdm

In [9]:
import warnings

warnings.simplefilter('ignore')

In [10]:
# mengambil informasi per hari
for url in tqdm(formatted_url):
    
    # untuk setiap harinya, kita perlu ambil jumlah pagination

    # Mengirim permintaan GET ke URL
    response = requests.get(url, verify=False)  # Menonaktifkan verifikasi SSL untuk contoh ini
    
    # Memastikan permintaan berhasil
    if response.status_code == 200:
        # Membuat objek lxml HTML dari respons
        tree = html.fromstring(response.content)
    else:
        print("Failed to retrieve the page. Status code:", response.status_code)


    # Jika tidak ada elemen <li>, print 1
    li_elements = tree.xpath('//*[@id="content-utama"]//div[@class="separate-content-list"]//div[@class="pagination"]/ul/li/a')
    
    if not li_elements:
        n_pagination = 1
    else:
        # Mencoba menemukan elemen "Next"
        next_element = tree.xpath('//*[@id="content-utama"]//div[@class="separate-content-list"]//div[@class="pagination"]/ul/li/a[text()="Next"]/..')
        
        if next_element:
            # Mendapatkan elemen sebelum "Next"
            prev_element = next_element[0].getprevious()
            if prev_element is not None:
                number = prev_element.xpath('.//text()')[0]
                n_pagination = number
            else:
                # Jika tidak ada elemen sebelum "Next", print 1
                n_pagination = 1
        else:
            # Jika tidak ada "Next", ambil angka dari elemen <li> terakhir
            last_number = li_elements[-1].xpath('.//text()')[0]
            n_pagination = last_number


    n_pagination = int(n_pagination)

    
    # sudah dapat pagination, lalu kita scrape tiap elemen per pagination
    for page in range(1, n_pagination+1):
        
        # URL yang ingin di-scrape
        url2 = f'{url}&page={page}'
    
        # Mengirim permintaan GET ke URL
        response = requests.get(url2, verify=False)  # Menonaktifkan verifikasi SSL untuk contoh ini
        
        # Memastikan permintaan berhasil
        if response.status_code == 200:
            # Membuat objek lxml HTML dari respons
            tree = html.fromstring(response.content)
        else:
            print("Failed to retrieve the page. Status code:", response.status_code)
    
        
        news_list = tree.xpath('//*[@id="content-utama"]//ul[@class="content-list"]/li')
        
        # Mencetak hasil
        for news in news_list:
            judul = "".join(news.xpath(".//h1/a/text()"))
            href = news.xpath(".//h1/a")[0].get('href')
            rubrik = "".join(news.xpath(".//h6/a/strong/text()"))
            tanggal = "".join(news.xpath(".//h6/a/span/text()"))
            
            # print(judul)
            # print(href)
            # print(rubrik)
            # print(tanggal)
            # print("")

            # Menyimpan data ke dalam database
            c.execute('''
            INSERT INTO news (judul, url, rubrik, tanggal)
            VALUES (?, ?, ?, ?)
            ''', (judul, href, rubrik, tanggal))
            
        # Commit perubahan setelah setiap halaman
        conn.commit()
        # print("Data dari halaman", url2, "tersimpan")
    
        # print("=======================")
        # print("")

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:17<00:00,  8.83s/it]


In [12]:
conn.close()